# Introduction
This kernel will explore the basics of image processing using a basic convolutional neural network running on tensor backend with keras as the programming interface. The MNIST image digits, popular as a beginner image data for introductory image processing will be used to train a convnet and make predictions.

## 1.0 Data exploration
Load and explore data

In [ ]:
#load libraries for data manipulation and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
# warnings
import string
import warnings
warnings.filterwarnings('ignore')

The data sets consist of grayscale image data in pixels in a structured format with each column representing a pixel of the image data.
* The training set has 42000 samples while the test set has 28000 samples
* The training set has 785 features while the test set has 784 features
* The training set has an extra variable 'label' which is the target variable

In [ ]:
# load the train and test data sets
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
print('Number of Training Samples = {}'.format(train.shape[0]))
print('Number of Test Samples = {}\n'.format(test.shape[0]))
print('Training X Shape = {}'.format(train.shape))
print('Training y Shape = {}\n'.format(train['label'].shape[0]))
print('Test X Shape = {}'.format(test.shape))
print('Test y Shape = {}\n'.format(test.shape[0]))
print('Index of Train Set:\n', train.columns)
print('Index of Test Set:\n', test.columns)

In [ ]:
# check datatypes
train.info()

The datatypes label to pixel783 consist of numeric integer64 values. 

In [ ]:
# sample of data
train.head()

In [ ]:
lp = sb.countplot(train['label'])

The label consist of 10 numeric values that are representations of digit images 0, 1, 2, 3, 4, 5, 6, 7, 8, and 9. The challenge is a multi-class classification of image data given its pixel values

In [ ]:
# check for missing values
missing_train = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_train = missing_train[missing_train > 0] * 100
print("There are {} train features with  missing values :".format(missing_train.shape[0]))
missing_test = (test.isnull().sum()/test.isnull().count()).sort_values(ascending=False)
missing_test= missing_test[missing_test > 0] * 100
print("There are {} test features with  missing values :".format(missing_test.shape[0]))


## 2.0 Recompose pixel digits to image data
The data sets are given as pixel digits, the image data can be reconstructed from the pixel digits. 784 pixels suggest the image is of size lenght 28 and width 28.(i.e. 28 x 28 = 784 pixels). 

In [ ]:
def show_image(train_image, train_label, index):
    image_shaped = train_image.values.reshape(28,28)
    plt.subplot(4, 6, index+1)
    plt.imshow(image_shaped, cmap=plt.cm.gray)
    plt.title(label)


plt.figure(figsize=(18, 8))
sample_image = train.sample(24).reset_index(drop=True)
for index, row in sample_image.iterrows():
    label = row['label']
    image_pixels = row.drop('label')
    show_image(image_pixels, label, index)
plt.tight_layout()

## 3.0 Data preparation
A convnet takes as input tensors in this format (sample, image_height, image_width, image_channels), hence we will configure the convnet to process inputs of size (28, 28, 1) where the channel is 1 representing a grayscale image. A coloured image in RGB colour format will have a channel value of 3.

The image values are transformed into a float32 array with values between 0 and 1 suitable for neural nets processing by dividing with 255.

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# prepare training and test sets
X = train.drop(columns=['label']).values.reshape((train.shape[0],28,28,1))
X_train = X.astype('float32') / 255
y = to_categorical(train['label'])
X_test = test.values.reshape((test.shape[0],28,28,1))
X_test = X_test.astype('float32') / 255

# prepare training and validation sets
train_images, test_images, train_labels, test_labels = train_test_split(X, y,
                                                test_size=0.1, random_state=0)
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

## 4.0 Modeling
Define a small convnet model, compile and evaluate model, evaluate model on test data

### 4.1 Defining a Small Convnet

In [ ]:
# defining a small convnet
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# adding a classifier on top of the convnet
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

### 4.2 Compile and Evaluate Model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
# model training 
model.compile(optimizer='rmsprop',
   loss='categorical_crossentropy',
   metrics=['acc'])
history = model.fit(train_images,
   train_labels,
   epochs=20,
   batch_size=64,
   callbacks=callbacks,
   validation_data=(test_images, test_labels))

The model acheived a validation accuracy of 98.9%

### 4.3 Evaluate Model on Test Data

In [ ]:
predictions = model.predict(X_test)
results = np.argmax(predictions, axis = 1) 

 Sample of Predictions

In [ ]:
plt.figure(figsize=(18, 8))
sample_test = test.head(24)
for index, image_pixels in sample_test.iterrows():
    label = results[index]
    show_image(image_pixels, label, index)
plt.tight_layout()

Creating submission file

In [ ]:
solution = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")
solution['Label'] = results
solution.to_csv('solution.csv', index = False)